In [1]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from rasterio.features import rasterize
from rasterio.enums import MergeAlg
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../")

import infrasap.market_access as ma
import infrasap.rasterMisc as rMisc
from infrasap.misc import tPrint

# Calculate portsheds

In [2]:
global_friction_surface = "/home/public/Data/GLOBAL/INFRA/FRICTION_2015/2015_friction_surface_v1_no_ocean_travel.tif"
ports_file = '/home/wb411133/data/Global/INFRA/PORTS/ports_20201207.shp'


In [3]:
inG = rasterio.open(global_friction_surface)
mcp = graph.MCP_Geometric(inG.read()[0,:,:])

In [5]:
inP = gpd.read_file(ports_file)
inP = inP.loc[inP['TYPE'] == "General"]
inP['IDX'] = [x+1 for x in list(inP.index)]

In [ ]:
importlib.reload(ma)

drive_time_thresholds = [0.25, 0.5, 1, 2, 5] # days
drive_time_thresholds = [x * 24 * 60 for x in drive_time_thresholds] #convert days to minutes
drive_vectors = ma.generate_feature_vectors(inG, mcp, inP, drive_time_thresholds, 'PORT')
drive_vectors.to_file(ports_file.replace(".shp", "_PORTSHEDS_20210128.shp"))

19:24:47	1 of 144: Ambon
19:33:25	2 of 144: Apia
19:41:48	3 of 144: Bangkok
19:54:19	4 of 144: Bar
20:07:46	5 of 144: Bitung
20:16:36	6 of 144: Dili
20:25:21	7 of 144: Gulhifalhu
20:34:09	8 of 144: Hazira (Adani)
20:46:35	9 of 144: Jakarta
20:55:18	10 of 144: Jayapura
21:03:58	11 of 144: Koper
21:17:42	12 of 144: Krishnapatnam (Adani)
21:29:32	13 of 144: Kupang
21:38:21	14 of 144: Laem Chabang
21:51:00	15 of 144: Lautoka
21:59:28	16 of 144: Merauke
22:08:04	17 of 144: Mundra (DP World)
22:20:34	18 of 144: Nuku'alofa
22:28:54	19 of 144: Palawan
22:37:31	20 of 144: Pipavav (APMT/Maersk)
22:49:37	21 of 144: Piraeus
23:02:39	22 of 144: Port Harcourt
23:12:13	23 of 144: Port Via
23:20:44	24 of 144: Semarang
23:29:24	25 of 144: Sittwe
23:41:42	26 of 144: Sorong
23:50:15	27 of 144: Surabaya
23:58:51	28 of 144: Suva
00:07:16	29 of 144: Thessaloniki
00:20:53	30 of 144: Tibar Bay
00:29:33	31 of 144: Trieste
00:43:05	32 of 144: Suape (Recife)
00:52:08	33 of 144: Pecém (near Fortaleza)
01:01:07	34

# Zonal Stats on VIIRS

In [11]:
portshed_file = ports_file.replace(".shp", "_PORTSHEDS_20210128.shp")
zonal_ntl_res = "/home/wb411133/temp/All_Port_VIIRS_ZONAL_Results.csv"
zonal_pop_area_res = "/home/wb411133/temp/All_Port_Pop_Area_Results.csv"
viirs_folder = "/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED"
viirs_vrts = []
for root, dirs, files in os.walk(viirs_folder):
    for f in files:
        if f[-4:] == ".vrt":
            viirs_vrts.append(os.path.join(root, f))

inD = gpd.read_file(portshed_file)
inD.head()

,threshold,IDX,geometry
0,360.0,Ambon,POLYGON ((128.2666666666667 -3.508333333333326...
1,720.0,Ambon,POLYGON ((128.2666666666667 -3.508333333333326...
2,1440.0,Ambon,POLYGON ((128.2666666666667 -3.508333333333326...
3,2880.0,Ambon,POLYGON ((128.2666666666667 -3.508333333333326...
4,7200.0,Ambon,POLYGON ((128.2666666666667 -3.508333333333326...


In [12]:
os.remove(zonal_ntl_res)

In [14]:
if not os.path.exists(zonal_ntl_res):
    all_res = {}
    for vrt in viirs_vrts:
        print(vrt)
        yr = os.path.basename(vrt).split("_")[0]
        res = rMisc.zonalStats(inD, vrt)
        res = pd.DataFrame(res, columns=["YR_%s_%s" % (yr, x) for x in ['SUM','MIN','MAX','MEAN']])
        all_res[yr] = res
else:
    print("Zonal file already exists")

/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2012_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2013_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2014_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2015_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2016_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2017_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2018_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2019_VIIRS_annual_composite.vrt


In [15]:
try:
    del final
except: 
    pass

for key, vals in all_res.items():
    vals['IDX'] = inD['threshold']
    xx = vals.iloc[:,[0,3]]
    try:        
        final = final.join(xx)
    except:
        final = xx

In [16]:
final['threshold'] = inD['threshold']
final['IDX'] = inD['IDX']
final.to_csv(zonal_ntl_res)

# Zonal stats on Pop

In [17]:
pop_file = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"
res = rMisc.zonalStats(inD, pop_file, minVal=0)
res = pd.DataFrame(res, columns=['pop_SUM', 'pop_MIN', 'pop_MAX', 'pop_MEAN'])

In [18]:
res['threshold'] = inD['threshold']
res['IDX'] = inD['IDX']

# Calculate area

In [20]:
marketsheds = inD
marketsheds = marketsheds.to_crs({'init':'epsg:6933'})

marketsheds['AREA_km'] = marketsheds['geometry'].apply(lambda x: x.area/1000000)

In [21]:
res['AREA_km'] = marketsheds['AREA_km']
res.to_csv(zonal_pop_area_res)

# Combine Results

In [22]:
inP = gpd.read_file(ports_file)
inP['IDX2'] = [x + 1 for x in list(inP.reset_index().index)]

In [23]:
inP.head()

,COUNTRY,IDX,PORT,REGION,TYPE,geometry,IDX2
0,Indonesia,1,Ambon,East Asia & Pacific,General,POINT (128.18333 -3.69583),1
1,Samoa,2,Apia,East Asia & Pacific,General,POINT (-171.76666 -13.83333),2
2,Thailand,3,Bangkok,East Asia & Pacific,General,POINT (100.50144 13.75398),3
3,Montenegro,4,Bar,Europe & Central Asia,General,POINT (19.09841 42.0937),4
4,Indonesia,5,Bitung,East Asia & Pacific,General,POINT (125.12824 1.44059),5


In [24]:
final['WP_2020_SUM'] = res['pop_SUM']
final['AREA'] = marketsheds['AREA_km']

In [25]:
final.head()

,YR_2012_SUM,YR_2012_MEAN,YR_2013_SUM,YR_2013_MEAN,YR_2014_SUM,YR_2014_MEAN,YR_2015_SUM,YR_2015_MEAN,YR_2016_SUM,YR_2016_MEAN,YR_2017_SUM,YR_2017_MEAN,YR_2018_SUM,YR_2018_MEAN,YR_2019_SUM,YR_2019_MEAN,threshold,IDX,WP_2020_SUM,AREA
0,1138.403931,0.387212,1234.422729,0.419872,1285.852417,0.437365,1576.5896,0.536255,1679.842529,0.571375,1917.655151,0.652264,1771.670654,0.602609,1987.227295,0.675928,360.0,Ambon,491814.5625,627.041085
1,1138.403931,0.354864,1234.422729,0.384795,1285.852417,0.400827,1576.5896,0.491456,1679.842529,0.523642,1917.655151,0.597773,1771.670654,0.552266,1987.227295,0.619460,720.0,Ambon,494896.8750,684.197320
2,1138.403931,0.354864,1234.422729,0.384795,1285.852417,0.400827,1576.5896,0.491456,1679.842529,0.523642,1917.655151,0.597773,1771.670654,0.552266,1987.227295,0.619460,1440.0,Ambon,494896.8750,684.197320
3,1138.403931,0.354864,1234.422729,0.384795,1285.852417,0.400827,1576.5896,0.491456,1679.842529,0.523642,1917.655151,0.597773,1771.670654,0.552266,1987.227295,0.619460,2880.0,Ambon,494896.8750,684.197320
4,1138.403931,0.354864,1234.422729,0.384795,1285.852417,0.400827,1576.5896,0.491456,1679.842529,0.523642,1917.655151,0.597773,1771.670654,0.552266,1987.227295,0.619460,7200.0,Ambon,494896.8750,684.197320


In [29]:
inP['IDX'] = inP['IDX'].astype(str)
final['IDX'] = final['IDX'].astype(str)

In [51]:
final_named = pd.merge(final, inP, left_on="IDX", right_on="PORT")
final_named.to_csv("/home/wb411133/temp/ALL_Port_Results.csv")

In [50]:
inP.head()

,COUNTRY,IDX,PORT,REGION,TYPE,geometry,IDX2
0,Indonesia,1,Ambon,East Asia & Pacific,General,POINT (128.18333 -3.69583),1
1,Samoa,2,Apia,East Asia & Pacific,General,POINT (-171.76666 -13.83333),2
2,Thailand,3,Bangkok,East Asia & Pacific,General,POINT (100.50144 13.75398),3
3,Montenegro,4,Bar,Europe & Central Asia,General,POINT (19.09841 42.0937),4
4,Indonesia,5,Bitung,East Asia & Pacific,General,POINT (125.12824 1.44059),5


In [49]:
final

,YR_2012_SUM,YR_2012_MEAN,YR_2013_SUM,YR_2013_MEAN,YR_2014_SUM,YR_2014_MEAN,YR_2015_SUM,YR_2015_MEAN,YR_2016_SUM,YR_2016_MEAN,YR_2017_SUM,YR_2017_MEAN,YR_2018_SUM,YR_2018_MEAN,YR_2019_SUM,YR_2019_MEAN,threshold,IDX,WP_2020_SUM,AREA
0,1.138404e+03,0.387212,1.234423e+03,0.419872,1.285852e+03,0.437365,1.576590e+03,0.536255,1.679843e+03,0.571375,1.917655e+03,0.652264,1.771671e+03,0.602609,1.987227e+03,0.675928,360.0,Ambon,4.918146e+05,6.270411e+02
1,1.138404e+03,0.354864,1.234423e+03,0.384795,1.285852e+03,0.400827,1.576590e+03,0.491456,1.679843e+03,0.523642,1.917655e+03,0.597773,1.771671e+03,0.552266,1.987227e+03,0.619460,720.0,Ambon,4.948969e+05,6.841973e+02
2,1.138404e+03,0.354864,1.234423e+03,0.384795,1.285852e+03,0.400827,1.576590e+03,0.491456,1.679843e+03,0.523642,1.917655e+03,0.597773,1.771671e+03,0.552266,1.987227e+03,0.619460,1440.0,Ambon,4.948969e+05,6.841973e+02
3,1.138404e+03,0.354864,1.234423e+03,0.384795,1.285852e+03,0.400827,1.576590e+03,0.491456,1.679843e+03,0.523642,1.917655e+03,0.597773,1.771671e+03,0.552266,1.987227e+03,0.619460,2880.0,Ambon,4.948969e+05,6.841973e+02
4,1.138404e+03,0.354864,1.234423e+03,0.384795,1.285852e+03,0.400827,1.576590e+03,0.491456,1.679843e+03,0.523642,1.917655e+03,0.597773,1.771671e+03,0.552266,1.987227e+03,0.619460,7200.0,Ambon,4.948969e+05,6.841973e+02
5,3.754822e+02,0.068770,3.950246e+02,0.072349,4.503635e+02,0.082484,4.597521e+02,0.084204,5.371167e+02,0.098373,6.691177e+02,0.122549,7.574277e+02,0.138723,9.983988e+02,0.182857,360.0,Apia,1.374535e+05,1.133340e+03
6,3.754822e+02,0.068770,3.950246e+02,0.072349,4.503635e+02,0.082484,4.597521e+02,0.084204,5.371167e+02,0.098373,6.691177e+02,0.122549,7.574277e+02,0.138723,9.983988e+02,0.182857,720.0,Apia,1.374535e+05,1.133340e+03
7,3.754822e+02,0.068770,3.950246e+02,0.072349,4.503635e+02,0.082484,4.597521e+02,0.084204,5.371167e+02,0.098373,6.691177e+02,0.122549,7.574277e+02,0.138723,9.983988e+02,0.182857,1440.0,Apia,1.374535e+05,1.133340e+03
8,3.754822e+02,0.068770,3.950246e+02,0.072349,4.503635e+02,0.082484,4.597521e+02,0.084204,5.371167e+02,0.098373,6.691177e+02,0.122549,7.574277e+02,0.138723,9.983988e+02,0.182857,2880.0,Apia,1.374535e+05,1.133340e+03
9,3.754822e+02,0.068770,3.950246e+02,0.072349,4.503635e+02,0.082484,4.597521e+02,0.084204,5.371167e+02,0.098373,6.691177e+02,0.122549,7.574277e+02,0.138723,9.983988e+02,0.182857,7200.0,Apia,1.374535e+05,1.133340e+03


# Generate overlap rasters

In [31]:
overlap_file = "/home/wb411133/temp/OVERLAP_FULL.tif"
adjusted_pop_file = "/home/wb411133/temp/new_pop_OVERLAP.tif"

In [32]:
drive_vectors = inD# gpd.read_file(ports_file.replace(".shp", "_PORTSHEDS.shp"))

In [33]:
thresh = 1440 # 24 hours
''' Subsetting for testing
sel_ports = [88, 89, 91]
& (drive_vectors['IDX'].isin(sel_ports))]
'''
inD = drive_vectors.loc[(drive_vectors['threshold'] == thresh)]
inD.reset_index(inplace=True)

In [34]:
# extract population for selected area
inR = rasterio.open(pop_file)
'''
out_file = "/home/wb411133/temp/TEST_POP_FILE.tif"
rMisc.clipRaster(inR, inD, out_file)
inR = rasterio.open(out_file)
'''

'\nout_file = "/home/wb411133/temp/TEST_POP_FILE.tif"\nrMisc.clipRaster(inR, inD, out_file)\ninR = rasterio.open(out_file)\n'

In [35]:
inR_data = inR.read()
port_shps = [[x['geometry'], 1] for idx, x in inD.iterrows()]

In [36]:
res = rasterize(port_shps, out_shape=[inR_data.shape[1], inR_data.shape[2]], 
                transform=inR.transform, merge_alg=MergeAlg.add)
res = res.astype(rasterio.uint8)

In [37]:
meta = inR.meta.copy()
meta.update(nodata=0, dtype=str(res.dtype))

with rasterio.open(overlap_file, 'w', **meta) as outR:
    outR.write_band(1, res)

In [38]:
# Divide the population layer by the overlap layer
new_pop = inR_data[0,:,:] / res

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [39]:
meta = inR.meta.copy()
meta.update(nodata=0, dtype=str(new_pop.dtype))

with rasterio.open(adjusted_pop_file, 'w', **meta) as outR:
    outR.write_band(1, new_pop)

In [43]:
pop_res = rMisc.zonalStats(inD, adjusted_pop_file, minVal=0)
overlap_res = rMisc.zonalStats(inD, overlap_file, unqVals=list(range(1,res.max() + 1)), rastType='C')

popD = pd.DataFrame(pop_res, columns = ['SUM','MIN','MAX','MEAN'])
popD['IDX'] = inD['IDX']

oD = pd.DataFrame(overlap_res, columns=['c%s' % x for x in list(range(1,res.max() + 1))])
oD['IDX'] = inD['IDX']

In [46]:
popD.to_csv(adjusted_pop_file.replace(".tif", "_%s_ZONAL.csv" % thresh))
oD.to_csv(overlap_file.replace(".tif", "_%s_ZONAL.csv" % thresh))